This is the test code for BC agent in the CARLA simulator.

In [1]:
import carla 
import math 
import random 
import time 
import csv
import numpy as np
import cv2
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing.image import array_to_img

2024-06-07 20:18:38.637665: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-07 20:18:38.687184: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-07 20:18:38.688872: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-07 20:18:39.587932: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
saved_model_path = 'Imitation_learning_trained_model.h5' 
loaded_model = tf.keras.models.load_model(saved_model_path)

2024-06-07 20:18:41.235973: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-07 20:18:41.236248: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-07 20:18:41.273900: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the 

In [3]:
# Connect the client and set up bp library and spawn points
client = carla.Client('localhost', 2000) 
world = client.get_world()
bp_lib = world.get_blueprint_library()  
spawn_points = world.get_map().get_spawn_points() 
print(spawn_points)

[<carla.libcarla.Transform object at 0x7be5e81b88c0>, <carla.libcarla.Transform object at 0x7be7986ef3c0>, <carla.libcarla.Transform object at 0x7be5e77b0cc0>, <carla.libcarla.Transform object at 0x7be5e77b0c40>, <carla.libcarla.Transform object at 0x7be5e77b0b40>, <carla.libcarla.Transform object at 0x7be5e77b0bc0>, <carla.libcarla.Transform object at 0x7be5e77b0dc0>, <carla.libcarla.Transform object at 0x7be5e77b0d40>, <carla.libcarla.Transform object at 0x7be5e77b0e40>, <carla.libcarla.Transform object at 0x7be5e77b0f40>, <carla.libcarla.Transform object at 0x7be5e775e040>, <carla.libcarla.Transform object at 0x7be5e775e0c0>, <carla.libcarla.Transform object at 0x7be5e775e140>, <carla.libcarla.Transform object at 0x7be5e775e1c0>, <carla.libcarla.Transform object at 0x7be5e775e240>, <carla.libcarla.Transform object at 0x7be5e775e2c0>, <carla.libcarla.Transform object at 0x7be5e775e340>, <carla.libcarla.Transform object at 0x7be5e77b0ec0>, <carla.libcarla.Transform object at 0x7be5e77

In [4]:
settings = world.get_settings()
settings.fixed_delta_seconds = 0.01
world.apply_settings(settings)

935

In [5]:
# Add the ego vehicle
vehicle_bp = bp_lib.find('vehicle.lincoln.mkz_2020') 
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[427])

# Move the spectator behind the vehicle to view it
spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4,z=2.5)),vehicle.get_transform().rotation) 
spectator.set_transform(transform)


In [6]:
print(spawn_points[1])

Transform(Location(x=584.831238, y=-13.577756, z=0.300000), Rotation(pitch=0.000000, yaw=-179.580566, roll=0.000000))


In [9]:
# Set initial camera translation
bound_x = 0.5 + vehicle.bounding_box.extent.x
bound_y = 0.5 + vehicle.bounding_box.extent.y
bound_z = 0.5 + vehicle.bounding_box.extent.z
camera_init_trans = carla.Transform(carla.Location(x=+0.8*bound_x, y=+0.0*bound_y, z=1.3*bound_z))

# Add one of each type of camera
camera_bp = bp_lib.find('sensor.camera.rgb') 
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

sem_camera_bp = bp_lib.find('sensor.camera.semantic_segmentation') 
sem_camera = world.spawn_actor(sem_camera_bp, camera_init_trans, attach_to=vehicle)


In [10]:
# Define respective callbacks
def rgb_callback(image, data_dict):
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

def sem_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))


In [11]:
def plot_image(model, image ):

    ## resize the input image 
    image = image[:,:,0:3]
    crop_coordinates = (350,719 , 0, 1279)

    # Crop the image using array slicing
    cropped_img = image[crop_coordinates[0]:crop_coordinates[1], crop_coordinates[2]:crop_coordinates[3]]
    image = cv2.resize(cropped_img, (160,160))
    image = np.array(image).astype("float32")/255.0
    image = np.reshape(image,(160,160,3))
    image = np.expand_dims(image, axis=0)
    X_train = image
    output= model.predict(X_train)

    return output

In [12]:
# Initialise parameters and data
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()



sensor_data = {'rgb_image': np.zeros((image_h, image_w, 4)),
               'sem_image': np.zeros((image_h, image_w, 4))}

# OpenCV named window for display
cv2.namedWindow('All cameras', cv2.WINDOW_AUTOSIZE)

# Tile all data in one array
top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image']),axis=1)#, sensor_data['inst_image']), axis=1)

# Display with imshow
cv2.imshow('All cameras',top_row)
cv2.waitKey(1)

# Set sensors recording
camera.listen(lambda image: rgb_callback(image, sensor_data))
sem_camera.listen(lambda image: sem_callback(image, sensor_data))

# Indefinite while loop
v2 = 0
acc = 0
steering = 0
brake = 0
cnt = 0
loc = []
with open('location2.csv', 'w', newline='') as file_command:
    writer = csv.writer(file_command,lineterminator='\n',)
    writer.writerow(['X',"Y","Z","pitch","yaw","roll","speed",'waypoint_x','waypoint_y','waypoint_z','width'])
    file_command.close()
while True:
    
    # Tile camera images into one array
    top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image']),axis=1)#, sensor_data['inst_image']), axis=1)
    position = (50, 50)  # (x, y) coordinates

    # Specify the font type
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Specify the font scale (font size)
    font_scale = 0.5

    # Specify the font color in BGR format
    font_color = (0, 0, 0)  # Blue

    # Specify the line type
    line_type = 2

    # Add the text to the image
    cv2.putText(top_row, 'velocity is: '+ str(v2) + '   throttle is: ' + str(acc) + '     steering is '+ str(steering)+ '     brake is '+ str(brake), position, font, font_scale, font_color, line_type)
    cv2.imshow('All cameras',top_row)

    output_cmd = plot_image(loaded_model, sensor_data['rgb_image'])

    v = vehicle.get_velocity()
    tr = vehicle.get_transform()
    v2= 3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2)
    carla_map = world.get_map()
    waypoint = carla_map.get_waypoint(tr.location, project_to_road=True, lane_type=carla.LaneType.Driving)
    width = waypoint.lane_width

    loc.append([tr.location.x,tr.location.y,tr.location.z,tr.rotation.pitch,tr.rotation.yaw,tr.rotation.roll,v2,waypoint.transform.location.x,waypoint.transform.location.y,waypoint.transform.location.z,width])
    
    

    steering =float(output_cmd[0])

    if v2<30:
        acc=0.6
    else:
        acc=0
    
    vehicle.apply_control(carla.VehicleControl(throttle=acc, steer=steering))
    
    # Break loop if user presses q
    if cv2.waitKey(1) == ord('q'):
        # opening the csv file in 'w+' mode
        with open('location2.csv', 'a', newline='') as file:
            write = csv.writer(file)
            write.writerows(loc)
        break

# Stop sensors and destroy OpenCV window
camera.stop()
sem_camera.stop()


1/1 [==============================] - 0s 41ms/step
